In [1]:

import json
import os
import pickle
import sys
sys.path.append(os.path.abspath(os.path.join("..", "..", "code")))

import folium
import pandas as pd

from identify_watersheds import WatershedIdentifier
from utils import list_streamgage_files


DATA_DIR = os.path.join("..", "..", "data")
TIF_DIR = os.path.join(DATA_DIR, "gage_tiff_files")
files_df = list_streamgage_files(TIF_DIR)
files_df = files_df[files_df["latest"]]
files_df

,file,gage_id,timestamp,latest
6,11402000_30m_elv_2022_10_04__21_07_11.tif,11402000,2022-10-04 21:07:11,True
12,11318500_30m_elv_2022_10_04__21_06_04.tif,11318500,2022-10-04 21:06:04,True
15,11266500_30m_elv_2022_10_10__22_48_53.tif,11266500,2022-10-10 22:48:53,True
16,11208000_30m_elv_2022_10_04__21_04_47.tif,11208000,2022-10-04 21:04:47,True
0,11202710_30m_elv_2022_10_04__21_03_37.tif,11202710,2022-10-04 21:03:37,True
3,11189500_30m_elv_2022_10_04__21_02_43.tif,11189500,2022-10-04 21:02:43,True
17,11185500_30m_elv_2022_10_04__21_01_40.tif,11185500,2022-10-04 21:01:40,True


In [2]:
gages = pd.read_csv(os.path.join(DATA_DIR, "target_gages.csv"), encoding="utf-8")
gages = gages.set_index(["site_no"])
gages

,agency_cd,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,coord_datum_cd,...,peak_count_nu,qw_begin_date,qw_end_date,qw_count_nu,gw_begin_date,gw_end_date,gw_count_nu,sv_begin_date,sv_end_date,sv_count_nu
site_no,,,,,,,,,,,,,,,,,,,,,
11185500,USGS,KERN R NO 3 CN NR KERNVILLE CA,ST,355420.0,1182800.0,35.905504,-118.467586,M,F,NAD27,...,0.0,--,--,0.0,--,--,0.0,2008-05-13,2022-04-28,33.0
11189500,USGS,SF KERN R NR ONYX CA,ST,354415.0,1181022.0,35.737452,-118.173689,M,F,NAD27,...,101.0,1979-11-14,1980-08-05,4.0,--,--,0.0,1930-03-25,2022-08-30,1103.0
11202710,USGS,MF TULE R BL INTAKE AB SPRINGVILLE CA,ST,360941.0,1184231.0,36.161336,-118.709536,M,F,NAD27,...,29.0,--,--,0.0,--,--,0.0,1990-06-12,2022-04-27,137.0
11208000,USGS,MARBLE F KAWEAH R (R ONLY) A POTWISHA CAMP CA,ST,363119.0,1184754.0,36.521890,-118.799265,M,F,NAD27,...,53.0,1980-06-12,1980-10-08,5.0,--,--,0.0,1984-06-10,2021-11-16,62.0
11266500,USGS,MERCED R A POHONO BRIDGE NR YOSEMITE CA,ST,374301.0,1193955.0,37.716871,-119.666279,M,F,NAD27,...,105.0,1971-10-18,2004-06-10,17.0,--,--,0.0,1916-11-02,2022-08-10,827.0
11318500,USGS,SF MOKELUMNE R NR WEST POINT CA,ST,382206.0,1203240.0,38.368246,-120.545482,M,F,NAD27,...,88.0,--,--,0.0,--,--,0.0,1958-01-26,2022-05-10,191.0
11402000,USGS,SPANISH C AB BLACKHAWK C AT KEDDIE CA,ST,400011.0,1205712.0,40.002947,-120.954399,M,F,NAD27,...,88.0,1971-09-21,1972-09-06,13.0,--,--,0.0,1935-04-04,2022-09-07,563.0


In [ ]:
watersheds = dict()
for ix, row in files_df.iterrows():
    f, gage = row["file"], row["gage_id"]
    lat, lon = gages.loc[int(gage)]["dec_lat_va"], gages.loc[int(gage)]["dec_long_va"]
    tif_fp = os.path.join(TIF_DIR, f)
    watersheds[gage] = WatershedIdentifier(tif_fp, lat, lon, 1)

In [ ]:
{k: v.catchment_touches_grid_edge for k, v in watersheds.items()}

In [ ]:
{k: v.clipped_grid.bbox for k, v in watersheds.items()}

In [3]:
def map_streamgage(lat: float, lon: float, gage_name: str = None, zoom_start: int = 5):
    """Map a streamgage."""
    if gage_name is None:
        gage_name = "Streamgage"
    my_map = folium.Map(location=[lat, lon], zoom_start=zoom_start, width="80%", height="100%")
    folium.Marker(location=[lat, lon], 
                  radius=10, popup=f"{gage_name} ({lat}, {lon})").add_to(my_map)
    return my_map

In [4]:
ADJUSTED_LAT_LON = dict()


def map_streamgage_adjust_marker(lat: float, lon: float, lat_adjust: float, lon_adjust: float,
                                 gage_name: str, zoom_start: int = 5):
    """Map a streamgage's original coordinates and an adjusted location."""
    print(f"Streamgage {gage_name}")
    my_map = folium.Map(location=[lat, lon], zoom_start=zoom_start, width="80%", height="100%")
    print(f"Original lat, lon: {lat}, {lon}")
    folium.Marker(location=[lat, lon], 
                  radius=10, popup=f"{gage_name} ({lat}, {lon})").add_to(my_map)
    adjusted_lat, adjusted_lon = lat + lat_adjust, lon + lon_adjust
    print(f"Adjusted lat, lon: {adjusted_lat}, {adjusted_lon}")
    folium.Marker(location=[adjusted_lat, adjusted_lon], 
                  radius=10, popup=f"ADJUSTED {gage_name} ({adjusted_lat}, {adjusted_lon})",
                  icon=folium.Icon(color="red")).add_to(my_map)
    
    ADJUSTED_LAT_LON[str(gage_name)] = (adjusted_lat, adjusted_lon)
        
    return my_map

In [5]:
gage = gages.iloc[0]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.00465, lon_adjust = -0.00275,
                                 gage_name = gage.name, zoom_start=16)

Streamgage 11185500
Original lat, lon: 35.90550439, -118.467586
Adjusted lat, lon: 35.90085439, -118.470336


In [6]:
gage = gages.iloc[1]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.0002, lon_adjust = -0.000,
                             gage_name = gage.name, zoom_start=18)

Streamgage 11189500
Original lat, lon: 35.7374516, -118.173689
Adjusted lat, lon: 35.7372516, -118.173689


In [7]:
gage = gages.iloc[2]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.00009, lon_adjust = -0.000,
                             gage_name = gage.name, zoom_start=18)

Streamgage 11202710
Original lat, lon: 36.1613358, -118.7095358
Adjusted lat, lon: 36.1612458, -118.7095358


In [8]:
gage = gages.iloc[3]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.0000, lon_adjust = -0.000,
                             gage_name = gage.name, zoom_start=18)

Streamgage 11208000
Original lat, lon: 36.5218899, -118.7992652
Adjusted lat, lon: 36.5218899, -118.7992652


In [9]:
gage = gages.iloc[4]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.00043, lon_adjust = +0.0005,
                                 gage_name = gage.name, zoom_start=16)

Streamgage 11266500
Original lat, lon: 37.71687138, -119.6662788
Adjusted lat, lon: 37.71644138, -119.6657788


In [10]:
gage = gages.iloc[5]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.00015, lon_adjust = -0.000,
                             gage_name = gage.name, zoom_start=18)

Streamgage 11318500
Original lat, lon: 38.36824625, -120.5454823
Adjusted lat, lon: 38.36809625, -120.5454823


In [11]:
gage = gages.iloc[6]
map_streamgage_adjust_marker(gage["dec_lat_va"], gage["dec_long_va"], lat_adjust = -0.00025, lon_adjust = -0.000,
                             gage_name = gage.name, zoom_start=18)

Streamgage 11402000
Original lat, lon: 40.002947, -120.9543994
Adjusted lat, lon: 40.002697, -120.9543994


In [12]:
ADJUSTED_LAT_LON

{'11185500': (35.90085439, -118.470336),
 '11189500': (35.7372516, -118.173689),
 '11202710': (36.1612458, -118.7095358),
 '11208000': (36.5218899, -118.7992652),
 '11266500': (37.71644138, -119.6657788),
 '11318500': (38.36809625, -120.5454823),
 '11402000': (40.002697, -120.9543994)}

In [13]:
with open(os.path.join(DATA_DIR, "adjusted_lat_lon.json"), "w") as f:
    json.dump(ADJUSTED_LAT_LON, f)